<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-700)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('/content/drive/MyDrive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

array(['ACC', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS',
       'AMBUJACEM', 'APOLLOHOSP', 'ASIANPAINT', 'AUBANK', 'AUROPHARMA',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DLF', 'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD',
       'FEDERALBNK', 'GAIL', 'GILLETTE', 'GLAND', 'GODREJCP', 'GODREJIND',
       'GRASIM', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL',
       'INDIANB', 'INDIGO', 'INDUSINDBK', 'INDUSTOWER', 'INFY', 'IOC',
       'IRCTC', 'ITC', 'JINDALSTEL', 'JSWSTEEL', 'JUBLFOOD', 'KOTAKBANK',
       'LT', 'LTIM', 'LUPIN', 'M&M', 'MARICO', 'MARUTI', 'MCDOWELL-N',
       '

In [5]:
# update Nifty features

'''
update_stocks = ['RELIANCE','HDFCBANK','TCS','ICICIBANK','HINDUNILVR','INFY','SBIN','BHARTIARTL','BAJFINANCE','HCLTECH','ASIANPAINT','MARUTI','TITAN','M&M','TATACONSUM','HEROMOTOCO','NESTLEIND']

for stock_id in update_stocks:
  #stock_id = 'RELIANCE'
  column = 'Conviction'
  value = 'H'
  mypf.loc[mypf['Symbol'] == stock_id, column] = value

mypf.to_csv('/content/drive/MyDrive/data/stocks/Nifty100Scrips.csv',index=False)
'''


"\nupdate_stocks = ['RELIANCE','HDFCBANK','TCS','ICICIBANK','HINDUNILVR','INFY','SBIN','BHARTIARTL','BAJFINANCE','HCLTECH','ASIANPAINT','MARUTI','TITAN','M&M','TATACONSUM','HEROMOTOCO','NESTLEIND']\n\nfor stock_id in update_stocks:\n  #stock_id = 'RELIANCE'\n  column = 'Conviction'\n  value = 'H'\n  mypf.loc[mypf['Symbol'] == stock_id, column] = value\n\nmypf.to_csv('/content/drive/MyDrive/data/stocks/Nifty100Scrips.csv',index=False)\n"

In [6]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    117
1      7
Name: InPortfolio, dtype: int64

In [7]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [9]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    #tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-03-20 00:00:00+05:30,16988.0,^NSEI,-4.74,-0.03,-0.62,34.0,18812.0,15198.0,50.47,11.0
2023-03-27 00:00:00+05:30,16986.0,^NSEI,-4.61,-0.24,-0.49,37.0,18812.0,15198.0,50.53,11.0
2023-04-05 00:00:00+05:30,17557.0,^NSEI,-1.35,-0.20,-0.44,58.0,18812.0,15198.0,34.73,7.0
2023-04-13 00:00:00+05:30,17828.0,^NSEI,0.14,-0.02,-0.13,66.0,18812.0,15198.0,27.23,6.0
2023-04-21 00:00:00+05:30,17624.0,^NSEI,-0.98,0.01,-0.02,56.0,18812.0,15198.0,32.87,7.0


In [11]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev.drop([ 'Avg_Cost', 'Med_PE'], axis=1, inplace=True)

In [12]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 7


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
69,443.0,JUBLFOOD.NS,-14.40,-2.32,-0.45,53.0,903.0,422.0,95.63,104.0,1.0,1.0,L
63,1227.0,INFY.NS,-17.79,-0.60,-2.20,23.0,1898.0,1223.0,99.41,55.0,1.0,1.0,H
33,3456.0,DMART.NS,-10.38,-1.74,-0.09,50.0,5324.0,3022.0,81.15,54.0,1.0,1.0,L
88,40055.0,PAGEIND.NS,-8.12,-1.79,-0.32,65.0,53631.0,29040.0,55.21,34.0,1.0,1.0,L
14,5938.0,BAJFINANCE.NS,-8.70,-1.32,0.08,54.0,7901.0,5265.0,74.47,33.0,1.0,1.0,H
106,700.0,TATACONSUM.NS,-7.35,-0.93,-0.42,42.0,875.0,631.0,71.72,25.0,1.0,1.0,H
7,2882.0,ASIANPAINT.NS,-4.34,-1.29,0.39,60.0,3566.0,2576.0,69.09,24.0,1.0,1.0,H


In [13]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 6


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
62,139.0,INDUSTOWER.NS,-21.85,-2.25,-0.65,34.0,303.0,137.0,98.80,118.0,1.0,1.0,L
65,603.0,IRCTC.NS,-9.05,-1.21,-0.33,55.0,1164.0,373.0,70.92,93.0,1.0,1.0,L
15,220.0,BANDHANBNK.NS,-7.74,-1.67,-0.91,59.0,346.0,187.0,79.25,57.0,1.0,1.0,L
49,513.0,HDFCLIFE.NS,-4.94,-0.56,-0.20,52.0,757.0,461.0,82.43,48.0,1.0,1.0,L
17,1481.0,BATAINDIA.NS,-8.62,-1.97,-0.56,68.0,2173.0,1387.0,88.04,47.0,1.0,1.0,L
18,585.0,BERGEPAINT.NS,-1.25,-0.84,0.52,52.0,858.0,540.0,85.85,47.0,1.0,1.0,L


In [14]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 7


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
62,139.0,INDUSTOWER.NS,-21.85,-2.25,-0.65,34.0,303.0,137.0,98.80,118.0,1.0,1.0,L
17,1481.0,BATAINDIA.NS,-8.62,-1.97,-0.56,68.0,2173.0,1387.0,88.04,47.0,1.0,1.0,L
18,585.0,BERGEPAINT.NS,-1.25,-0.84,0.52,52.0,858.0,540.0,85.85,47.0,1.0,1.0,L
49,513.0,HDFCLIFE.NS,-4.94,-0.56,-0.20,52.0,757.0,461.0,82.43,48.0,1.0,1.0,L
26,915.0,CIPLA.NS,-12.10,-0.86,-1.19,53.0,1173.0,856.0,81.39,28.0,1.0,1.0,L
15,220.0,BANDHANBNK.NS,-7.74,-1.67,-0.91,59.0,346.0,187.0,79.25,57.0,1.0,1.0,L
65,603.0,IRCTC.NS,-9.05,-1.21,-0.33,55.0,1164.0,373.0,70.92,93.0,1.0,1.0,L


In [15]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 3


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
59,302.0,INDIANB.NS,13.63,2.63,0.70,57.0,322.0,112.0,9.52,7.0,1.0,1.0,L
16,176.0,BANKBARODA.NS,8.25,1.84,0.60,63.0,193.0,71.0,13.93,10.0,1.0,1.0,L
104,989.0,SUNPHARMA.NS,0.45,0.81,-0.31,50.0,1044.0,651.0,13.99,6.0,1.0,1.0,L


In [16]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
34,4869.0,DRREDDY.NS,10.33,0.91,1.19,73.0,5451.0,3669.0,32.66,12.0,1.0,1.0,L


In [17]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 9


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
113,2568.0,TITAN.NS,0.96,-0.12,0.85,59.0,2770.0,1512.0,16.06,8.0,1.0,1.0,H
18,585.0,BERGEPAINT.NS,-1.25,-0.84,0.52,52.0,858.0,540.0,85.85,47.0,1.0,1.0,L
92,2440.0,PIDILITIND.NS,-3.36,-1.15,0.47,66.0,2902.0,1991.0,50.71,19.0,1.0,1.0,L
35,3201.0,EICHERMOT.NS,-3.62,-0.51,-0.03,59.0,3850.0,2241.0,40.34,20.0,1.0,1.0,L
49,513.0,HDFCLIFE.NS,-4.94,-0.56,-0.20,52.0,757.0,461.0,82.43,48.0,1.0,1.0,L
65,603.0,IRCTC.NS,-9.05,-1.21,-0.33,55.0,1164.0,373.0,70.92,93.0,1.0,1.0,L
17,1481.0,BATAINDIA.NS,-8.62,-1.97,-0.56,68.0,2173.0,1387.0,88.04,47.0,1.0,1.0,L
62,139.0,INDUSTOWER.NS,-21.85,-2.25,-0.65,34.0,303.0,137.0,98.80,118.0,1.0,1.0,L
15,220.0,BANDHANBNK.NS,-7.74,-1.67,-0.91,59.0,346.0,187.0,79.25,57.0,1.0,1.0,L


In [18]:
# all time high sales and profit TTM

star_stocks = ['ADANIENT.NS', 'DRREDDY.NS', 'SBIN.NS', 'HCLTECH.NS', 'HDFC.NS',
       'HDFCBANK.NS', 'SBICARD.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'IGL.NS',
       'RELIANCE.NS', 'INDUSINDBK.NS', 'ADANIGREEN.NS', 'INFY.NS', 'ITC.NS',
       'PIIND.NS', 'PIDILITIND.NS', 'JUBLFOOD.NS', 'PAGEIND.NS', 'LTIM.NS',
       'M&M.NS', 'MARICO.NS', 'MCDOWELL-N.NS', 'EICHERMOT.NS', 'GRASIM.NS',
       'DMART.NS', 'TCS.NS', 'UPL.NS', 'ASIANPAINT.NS', 'AXISBANK.NS',
       'TITAN.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BANDHANBNK.NS',
       'BHARTIARTL.NS', 'BERGEPAINT.NS', 'COALINDIA.NS', 'CHOLAFIN.NS',
       'TATACONSUM.NS', 'CIPLA.NS', 'NTPC.NS', 'BRITANNIA.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 18


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
1,1803.0,ADANIENT.NS,-39.74,-3.41,-2.17,48.0,4165.0,1194.0,79.50,131.0,1.0,0.0,L
69,443.0,JUBLFOOD.NS,-14.40,-2.32,-0.45,53.0,903.0,422.0,95.63,104.0,1.0,1.0,L
72,4171.0,LTIM.NS,-8.85,-0.02,-0.47,29.0,7439.0,3566.0,84.38,78.0,1.0,0.0,L
15,220.0,BANDHANBNK.NS,-7.74,-1.67,-0.91,59.0,346.0,187.0,79.25,57.0,1.0,1.0,L
63,1227.0,INFY.NS,-17.79,-0.60,-2.20,23.0,1898.0,1223.0,99.41,55.0,1.0,1.0,H
33,3456.0,DMART.NS,-10.38,-1.74,-0.09,50.0,5324.0,3022.0,81.15,54.0,1.0,1.0,L
99,762.0,SBICARD.NS,-4.11,-1.51,0.40,58.0,1133.0,673.0,80.65,49.0,1.0,0.0,L
18,585.0,BERGEPAINT.NS,-1.25,-0.84,0.52,52.0,858.0,540.0,85.85,47.0,1.0,1.0,L
12,1318.0,BAJAJFINSV.NS,-12.72,-1.66,0.15,51.0,1907.0,1093.0,72.36,45.0,1.0,0.0,L
88,40055.0,PAGEIND.NS,-8.12,-1.79,-0.32,65.0,53631.0,29040.0,55.21,34.0,1.0,1.0,L


In [19]:
# star stocks ordered based on discount
tmp_df.sort_values(by = 'Discount%', ascending=False)

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
63,1227.0,INFY.NS,-17.79,-0.60,-2.20,23.0,1898.0,1223.0,99.41,55.0,1.0,1.0,H
69,443.0,JUBLFOOD.NS,-14.40,-2.32,-0.45,53.0,903.0,422.0,95.63,104.0,1.0,1.0,L
18,585.0,BERGEPAINT.NS,-1.25,-0.84,0.52,52.0,858.0,540.0,85.85,47.0,1.0,1.0,L
72,4171.0,LTIM.NS,-8.85,-0.02,-0.47,29.0,7439.0,3566.0,84.38,78.0,1.0,0.0,L
26,915.0,CIPLA.NS,-12.10,-0.86,-1.19,53.0,1173.0,856.0,81.39,28.0,1.0,1.0,L
33,3456.0,DMART.NS,-10.38,-1.74,-0.09,50.0,5324.0,3022.0,81.15,54.0,1.0,1.0,L
99,762.0,SBICARD.NS,-4.11,-1.51,0.40,58.0,1133.0,673.0,80.65,49.0,1.0,0.0,L
1,1803.0,ADANIENT.NS,-39.74,-3.41,-2.17,48.0,4165.0,1194.0,79.50,131.0,1.0,0.0,L
15,220.0,BANDHANBNK.NS,-7.74,-1.67,-0.91,59.0,346.0,187.0,79.25,57.0,1.0,1.0,L
110,3161.0,TCS.NS,-2.02,0.15,-0.91,45.0,3886.0,2915.0,74.67,23.0,1.0,0.0,H


In [20]:
# star stocks ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
18,585.0,BERGEPAINT.NS,-1.25,-0.84,0.52,52.0,858.0,540.0,85.85,47.0,1.0,1.0,L
99,762.0,SBICARD.NS,-4.11,-1.51,0.40,58.0,1133.0,673.0,80.65,49.0,1.0,0.0,L
7,2882.0,ASIANPAINT.NS,-4.34,-1.29,0.39,60.0,3566.0,2576.0,69.09,24.0,1.0,1.0,H
12,1318.0,BAJAJFINSV.NS,-12.72,-1.66,0.15,51.0,1907.0,1093.0,72.36,45.0,1.0,0.0,L
14,5938.0,BAJFINANCE.NS,-8.70,-1.32,0.08,54.0,7901.0,5265.0,74.47,33.0,1.0,1.0,H
97,2349.0,RELIANCE.NS,-4.32,-0.65,0.04,55.0,2811.0,1951.0,53.72,20.0,1.0,0.0,H
35,3201.0,EICHERMOT.NS,-3.62,-0.51,-0.03,59.0,3850.0,2241.0,40.34,20.0,1.0,1.0,L
33,3456.0,DMART.NS,-10.38,-1.74,-0.09,50.0,5324.0,3022.0,81.15,54.0,1.0,1.0,L
88,40055.0,PAGEIND.NS,-8.12,-1.79,-0.32,65.0,53631.0,29040.0,55.21,34.0,1.0,1.0,L
106,700.0,TATACONSUM.NS,-7.35,-0.93,-0.42,42.0,875.0,631.0,71.72,25.0,1.0,1.0,H


In [21]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 42


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
4,995.0,ADANITRANS.NS,-56.63,-8.41,-6.42,46.0,4105.0,643.0,89.83,313.0,1.0,-1.0,L
89,718.0,PEL.NS,-12.07,-3.30,-1.73,53.0,2866.0,635.0,96.28,299.0,-1.0,0.0,L
54,57.0,IBREALEST.NS,-20.90,-2.78,-1.75,51.0,191.0,47.0,93.06,235.0,-1.0,-1.0,L
40,1311.0,GLAND.NS,-18.86,-4.76,0.31,54.0,4316.0,1154.0,95.03,229.0,1.0,0.0,L
2,916.0,ADANIGREEN.NS,-42.77,-5.93,-2.23,54.0,2970.0,462.0,81.90,224.0,-1.0,-1.0,L
1,1803.0,ADANIENT.NS,-39.74,-3.41,-2.17,48.0,4165.0,1194.0,79.50,131.0,1.0,0.0,L
84,54.0,NETWORK18.NS,-13.97,-1.59,-1.75,44.0,114.0,40.0,81.08,111.0,-1.0,-1.0,L
120,368.0,WIPRO.NS,-5.87,-0.77,-1.05,47.0,713.0,356.0,96.64,94.0,-1.0,0.0,L
82,3627.0,NAUKRI.NS,-4.49,-0.92,-0.06,49.0,6984.0,3363.0,92.71,93.0,1.0,0.0,L
119,1322.0,WHIRLPOOL.NS,-9.62,-2.12,-0.08,48.0,2510.0,1254.0,94.59,90.0,1.0,0.0,L


In [22]:
stock_id = 'VOLTAS.NS'
df_prec_dev[df_prec_dev['Stock'] == stock_id]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
118,857.0,VOLTAS.NS,0.94,-1.06,0.58,56.0,1330.0,745.0,80.85,55.0,0.0,1.0,L
